In [3]:
!pip install spacy -q
!python -m spacy info


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip

============================== Info about spaCy ==============================

spaCy version    3.7.4                         
Location         /usr/local/lib/python3.10/dist-packages/spacy
Platform         Linux-6.2.0-36-generic-x86_64-with-glibc2.35
Python version   3.10.12                       
Pipelines                                      



In [4]:
import json

In [5]:
with open('annotations.json', 'r') as f:
    json_data = f.read()
    data = json.loads(json_data)

In [8]:
training_data: list = []

for line in data:
    training_data.append(  [ line[0], line[1]] )
print(len(training_data))

201


In [9]:
line = training_data[0][1]

print(line)

{'entities': [[553, 560, 'MONEY'], [622, 623, 'YEARS']]}


In [10]:
train_split = int(len(training_data) * 0.8) # 80% training and 20% deve set

train_data = training_data[:train_split]
dev_data = training_data[train_split:]

In [11]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm


def convert(path, dataset):
    nlp = spacy.blank("en")
    db = DocBin()
    for text, annot in tqdm(dataset): 
            doc = nlp.make_doc(text) 
            ents = []
            for start, end, label in annot["entities"]:
                span = doc.char_span(start, end, label=label, alignment_mode="contract")
                if span is None:
                    print("Skipping nil entity")                
                if span.text != span.text.strip():
                    print("Skipping entity spans with whitespace")
                else:
                    ents.append(span)
            doc.ents = ents

            db.add(doc)
    db.to_disk(path)
    
convert("train.spacy", train_data)
convert("dev.spacy", dev_data)

100%|██████████| 41/41 [00:00<00:00, 1367.70it/s]


In [12]:
!python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency --force

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [62]:
!spacy debug data config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


============================ Data file validation ============================
✔ Pipeline can be initialized with data
✔ Corpus is loadable

=============================== Training stats ===============================
Language: en
Training pipeline: tok2vec, ner
160 training docs
40 evaluation docs
✔ No overlap between training and evaluation data
⚠ Low number of examples to train a new pipeline (160)

============================== Vocab & Vectors ==============================
ℹ 48250 total word(s) in the data (2104 unique)
ℹ No word vectors present in the package

========================== Named Entity Recognition ==========================
ℹ 2 label(s)
0 missing value(s) (tokens with '-' label)
✔ Good amount of examples for all labels
✔ Examples without occurrences available for all labels
✔ No entities consisting of or starting/ending with whitespace
✔ No entities crossing sentence boundaries

================================== Summary ==================================
✔ 7 ch

In [64]:
!python -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./dev.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     76.90    0.00    0.00    0.00    0.00
  1     200         27.48   2793.98   72.18   73.85   70.59    0.72
  2     400         43.17    294.32   74.29   72.22   76.47    0.74
  3     600         72.66    259.31   72.59   73.13   72.06    0.73
  5     800       1456.35    409.43   75.82   68.24   85.29    0.76
  6    1000        122.64    184.28   73.76   71.23   76.47    0.74
  7    1200        145.98    187.42   70.59   70.59   70.59    0.71
  8    1400        144.71    167.94   75.71   73.61   77.94    0.76
 10    1600        160.73    159.93 

In [18]:
text = "section ii. award information funding instrument grant: a support mechanism providing money, property, or both to an eligible entity to carry out an approved project or activity. application types allowed new resubmissions the oer glossary and the sf424 (r&r) application guide provide details on these application types. clinical trial optional: accepting applications that either propose or do not propose clinical trial(s) need help determining whether you are doing a clinical trial? funds available and anticipated number of awards nih anticipates providing $5m per year to fund an estimated 5 to 10 awards award budget application budgets are not limited but need to reflect the actual needs of the proposed project. award project period the scope of the proposed project should determine the project period. the maximum project period is 5 years. nih grants policies as described in the nih grants policy statement will apply to the applications submitted and awards made from this foa."
nlp = spacy.load("model-best")
doc = nlp(text)
for ent in doc.ents:
#print(ent.label_, ent.text)
    print(ent.text)

5m
5


In [72]:
colors = {'MONEY': "#FFFF33",'YEARS': "#FF006E" }
options = {"ents": ['MONEY', 'YEARS'], "colors": colors}

spacy.displacy.render(doc, style="ent", jupyter=True, options=options)